In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


# **PREPARING THE DATASETS**

In [ ]:
#import file:
customer = '/content/drive/My Drive/data_domain/dim_customer.csv'
product = '/content/drive/My Drive/data_domain/dim_product.csv'
gross_price = '/content/drive/My Drive/data_domain/fact_gross_price.csv'
cost = '/content/drive/My Drive/data_domain/fact_manufacturing_cost.csv'
discount = '/content/drive/My Drive/data_domain/fact_pre_discount.csv'
sale = '/content/drive/My Drive/data_domain/fact_sales_monthly.csv'

In [ ]:
all = ['customer','product','price','cost','discount','sale']
for name in all:
  name = pd.read_csv(name)

In [ ]:
customer = pd.read_csv(customer)
product = pd.read_csv(product)
price = pd.read_csv(gross_price)
cost = pd.read_csv(cost)
discount = pd.read_csv(discount)
sale = pd.read_csv(sale)

In [ ]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_code  209 non-null    int64 
 1   customer       209 non-null    object
 2   platform       209 non-null    object
 3   channel        209 non-null    object
 4   market         209 non-null    object
 5   sub_zone       183 non-null    object
 6   region         183 non-null    object
dtypes: int64(1), object(6)
memory usage: 11.6+ KB


In [ ]:
customer.columns

Index(['customer_code', 'customer', 'platform', 'channel', 'market',
       'sub_zone', 'region'],
      dtype='object')

As we don't need sub_zone and region column so they will be deleted in merging step

In [ ]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  397 non-null    object
 1   division      397 non-null    object
 2   segment       397 non-null    object
 3   category      397 non-null    object
 4   product       397 non-null    object
 5   variant       397 non-null    object
dtypes: object(6)
memory usage: 18.7+ KB


In [ ]:
cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_code        1182 non-null   object 
 1   cost_year           1182 non-null   int64  
 2   manufacturing_cost  1182 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 27.8+ KB


In [ ]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_code  1182 non-null   object 
 1   fiscal_year   1182 non-null   int64  
 2   gross_price   1182 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 27.8+ KB


In [ ]:
discount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_code             1045 non-null   int64  
 1   fiscal_year               1045 non-null   int64  
 2   pre_invoice_discount_pct  1045 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 24.6 KB


In [ ]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67251 entries, 0 to 67250
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           67251 non-null  object 
 1   product_code   67251 non-null  object 
 2   customer_code  67250 non-null  float64
 3   sold_quantity  67250 non-null  float64
 4   fiscal_year    67250 non-null  float64
dtypes: float64(3), object(2)
memory usage: 2.6+ MB


The datatype of float should be convert to int by the column context

In [ ]:
sale[['customer_code','sold_quantity','fiscal_year']] = sale[['customer_code','sold_quantity','fiscal_year']].astype(int)

In [ ]:
sale_nan = sale[sale.isna().any(axis=1)]
sale_nan

,date,product_code,customer_code,sold_quantity,fiscal_year
67250,2019-06-01,A0,NaN,NaN,NaN


We have to delete this product_code to qualify the data



In [ ]:
sale = sale[sale['product_code']!='A0']

In [ ]:
sale.isnull().sum()

date             0
product_code     0
customer_code    0
sold_quantity    0
fiscal_year      0
dtype: int64

In [ ]:
customer['customer'].value_counts()

Amazon             25
Atliq e Store      24
Atliq Exclusive    17
Expert              5
Euronics            4
                   ..
Croma               1
Ezone               1
Vijay Sales         1
Lotus               1
Saturn              1
Name: customer, Length: 75, dtype: int64

In [ ]:
customer.loc[customer['customer']== 'Amazon ', 'customer'] = 'Amazon'

# **MERGING DATA**

In [ ]:
#JOIN sale and price tables
df = sale.merge(price[['product_code','fiscal_year','gross_price']], on = ['product_code','fiscal_year'])
df.head()

,date,product_code,customer_code,sold_quantity,fiscal_year,gross_price
0,2017-09-01,A0118150101,70002017,51,2018,15.3952
1,2017-09-01,A0118150101,70002018,77,2018,15.3952
2,2017-09-01,A0118150101,70003181,17,2018,15.3952
3,2017-09-01,A0118150101,70003182,6,2018,15.3952
4,2017-09-01,A0118150101,70006157,5,2018,15.3952


In [ ]:
#JOIN with customer table
df = df.merge(customer[['customer_code', 'customer', 'platform', 'channel', 'market']], on = ['customer_code'])
df.head()

,date,product_code,customer_code,sold_quantity,fiscal_year,gross_price,customer,platform,channel,market
0,2017-09-01,A0118150101,70002017,51,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India
1,2017-10-01,A0118150101,70002017,54,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India
2,2017-11-01,A0118150101,70002017,30,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India
3,2018-01-01,A0118150101,70002017,14,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India
4,2018-02-01,A0118150101,70002017,62,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India


In [ ]:
#JOIN with discount table
df = df.merge(discount[['customer_code', 'fiscal_year', 'pre_invoice_discount_pct']], on = ['customer_code', 'fiscal_year'])
df.rename(columns = {'pre_invoice_discount_pct':'discount'},inplace=True)
df.head()

,date,product_code,customer_code,sold_quantity,fiscal_year,gross_price,customer,platform,channel,market,discount
0,2017-09-01,A0118150101,70002017,51,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824
1,2017-10-01,A0118150101,70002017,54,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824
2,2017-11-01,A0118150101,70002017,30,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824
3,2018-01-01,A0118150101,70002017,14,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824
4,2018-02-01,A0118150101,70002017,62,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824


In [ ]:
#JOIN with cost table
cost.rename(columns={'cost_year':'fiscal_year'},inplace=True)
df = df.merge(cost[['product_code', 'fiscal_year', 'manufacturing_cost']], on = ['product_code','fiscal_year'])
df.rename(columns={'manufacturing_cost':'cost'},inplace=True)
df.head()

,date,product_code,customer_code,sold_quantity,fiscal_year,gross_price,customer,platform,channel,market,discount,cost
0,2017-09-01,A0118150101,70002017,51,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619
1,2017-10-01,A0118150101,70002017,54,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619
2,2017-11-01,A0118150101,70002017,30,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619
3,2018-01-01,A0118150101,70002017,14,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619
4,2018-02-01,A0118150101,70002017,62,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619


In [ ]:
#JOIN with product table
df = df.merge(product[['product_code', 'division', 'segment', 'category', 'product','variant']],on='product_code')
df.head()

,date,product_code,customer_code,sold_quantity,fiscal_year,gross_price,customer,platform,channel,market,discount,cost,division,segment,category,product,variant
0,2017-09-01,A0118150101,70002017,51,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
1,2017-10-01,A0118150101,70002017,54,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
2,2017-11-01,A0118150101,70002017,30,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
3,2018-01-01,A0118150101,70002017,14,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
4,2018-02-01,A0118150101,70002017,62,2018,15.3952,Atliq Exclusive,Brick & Mortar,Direct,India,0.0824,4.619,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard


In [ ]:
df.isnull().sum()

date             0
product_code     0
customer_code    0
sold_quantity    0
fiscal_year      0
gross_price      0
customer         0
platform         0
channel          0
market           0
discount         0
cost             0
division         0
segment          0
category         0
product          0
variant          0
dtype: int64

In [ ]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
67245    False
67246    False
67247    False
67248    False
67249    False
Length: 67250, dtype: bool

In [ ]:
from google.colab import files
df.to_csv('dataset.csv') 
files.download('dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>